## *Rodar notebook toda quinta-feira já que às quintas os atletas têm suas habilidades atualizadas após o treino*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]
print("Data:", agora)

Data: 2023-12-09 11:11:04


In [3]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


Como meu clube não pode ter valores nulos, os dos outros usuários sim (não tenho acesso a alguns dados de seus jogadores), eu insiro uma cláusula if antes de iniciar a inserção dos dados na base de dados para conferir.

In [4]:
# URL da API
url= f'https://sokker.org/xml/players-{club_id}.xml'

# Pegando os dados dos atletas
df_players= pega_dados(url, sessao, search= 'player')

# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_players= df_players.assign(date_extracted= agora)

# Soma de valores nulos ou duplicados
nulos= df_players.isnull().sum().sum()
duplicados=  df_players.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_players.to_sql('players', con=db, if_exists='append', index= False) 
    print('Dados extraídos com sucesso!')

Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...
Dados extraídos com sucesso!


In [5]:
df = pd.read_sql_query("SELECT * FROM players", db)

In [6]:
df

,ID,date_extracted,name,surname,countryID,age,teamID,wage,height,weight,...,skillDiscipline,skillPace,skillTechnique,skillPassing,skillKeeper,skillDefending,skillPlaymaking,skillScoring,trainingPosition,isInTrainingSlot
0,36519432,2021-07-27 14:44:35,Krastan,Apostolov,54,29,79085,33900.0,177.0,71.5,...,5,15,3,9,10,5,5,3,None,None
1,37968035,2021-07-27 14:44:35,Romildo,Barbadiano,21,29,79085,3900.0,171.0,63.9,...,1,5,8,3,0,8,1,3,None,None
2,37968034,2021-07-27 14:44:35,Giovani,Barriguinha,21,27,79085,2600.0,172.0,76.3,...,0,6,2,5,1,5,1,4,None,None
3,36103236,2021-07-27 14:44:35,Ilario,Bartolozzi,10,29,79085,16900.0,169.0,67.3,...,9,13,7,8,0,8,9,8,None,None
4,37968028,2021-07-27 14:44:35,Cecilio,Bazzarella,21,20,79085,4800.0,175.0,77.4,...,1,7,1,9,0,5,7,7,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2951,38737051,2023-12-09 11:11:04,Luk,Schockaert,28,22,79085,16300.0,186.0,87.5,...,5,10,7,7,1,12,6,6,1,true
2952,37773691,2023-12-09 11:11:04,Sergiu,Socea,9,29,79085,29300.0,180.0,89.3,...,15,12,13,3,0,9,4,11,3,false
2953,37857538,2023-12-09 11:11:04,Julian,Stosik,1,28,79085,34800.0,169.0,64.4,...,9,12,7,5,13,5,6,6,0,false
2954,37276405,2023-12-09 11:11:04,Artur,Szymkiewicz,1,31,79085,27000.0,179.0,75.3,...,15,14,11,8,0,6,9,9,2,false


In [7]:
len(df)

2956

## 

In [8]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()